Working with CoefficientFunctions
====

In [1]:
import netgen.gui
%gui tk
from netgen.geom2d import unit_square
from ngsolve import *
mesh = Mesh (unit_square.GenerateMesh(maxh=0.2))

Define a function a plot it in the Netgen-gui:

In [2]:
myfunc = x*(1-x)
Draw (myfunc, mesh, "first")

We can (numerically) intgrate the function on the mesh:

In [3]:
Integrate (myfunc, mesh)

0.16666666666666602

In [4]:
mip = mesh(0.2, 0.2)
myfunc(mip)

0.16000000000000003

In [5]:
pts = [(0.1*i, 0.2) for i in range(10)]
vals = [myfunc(mesh(*p)) for p in pts] 
for p,v in zip(pts, vals):
    print ("point =",p,"val =",v)

point = (0.0, 0.2) val = 0.0
point = (0.1, 0.2) val = 0.09000000000000002
point = (0.2, 0.2) val = 0.16000000000000003
point = (0.30000000000000004, 0.2) val = 0.21000000000000002
point = (0.4, 0.2) val = 0.24
point = (0.5, 0.2) val = 0.25
point = (0.6000000000000001, 0.2) val = 0.24
point = (0.7000000000000001, 0.2) val = 0.20999999999999996
point = (0.8, 0.2) val = 0.15999999999999998
point = (0.9, 0.2) val = 0.08999999999999998
